In [ ]:
import numpy as np
import pandas as pd
from scapy.all import sniff, IP, TCP, UDP
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import time
import threading
import os
import iptc  # For managing iptables firewall

# Data structure to store traffic
traffic_data = []

# Function to capture packets
def packet_callback(packet):
    if IP in packet:
        ip_src = packet[IP].src
        ip_dst = packet[IP].dst
        packet_length = len(packet)
        protocol = packet.proto
        
        # Capture relevant traffic details
        packet_data = {
            'src_ip': ip_src,
            'dst_ip': ip_dst,
            'length': packet_length,
            'protocol': protocol,
            'time': time.time()
        }
        traffic_data.append(packet_data)

# Traffic Sniffing
def start_sniffing():
    sniff(prn=packet_callback, store=0, timeout=60)  # Sniff for 60 seconds

# Feature extraction and preprocessing
def extract_features(data):
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['time_diff'] = df['time'].diff().fillna(0).dt.total_seconds()
    
    # Calculate traffic rate (packets per second)
    df['traffic_rate'] = df.groupby('src_ip')['time_diff'].transform('sum') / df['time_diff'].count()
    
    # Standardize features
    df['length'] = StandardScaler().fit_transform(df[['length']])
    df['traffic_rate'] = StandardScaler().fit_transform(df[['traffic_rate']])
    
    return df

# Train DDoS Detection Model
def train_ddos_model(data):
    df = extract_features(data)
    X = df[['length', 'traffic_rate']].values
    y = df['protocol'].apply(lambda x: 1 if x == 6 else 0)  # Classify TCP as attack, others as normal

    model = RandomForestClassifier(n_estimators=100)
    model.fit(X, y)
    
    return model

# Anomaly Detection using Random Forest
def detect_ddos(model, data):
    df = extract_features(data)
    X = df[['length', 'traffic_rate']].values
    predictions = model.predict(X)
    
    return predictions

# Block Malicious IP
def block_ip(ip_address):
    os.system(f"iptables -A INPUT -s {ip_address} -j DROP")
    print(f"Blocked IP: {ip_address}")

# Real-Time DDoS Detection and Mitigation
def monitor_and_block_ddos():
    while True:
        if len(traffic_data) > 100:
            # Train model after collecting sufficient data
            model = train_ddos_model(traffic_data)
            
            # Detect anomalies (DDoS)
            predictions = detect_ddos(model, traffic_data)
            
            # If DDoS detected (prediction = 1 for attack)
            for i, prediction in enumerate(predictions):
                if prediction == 1:
                    blocked_ip = traffic_data[i]['src_ip']
                    print(f"DDoS detected from IP: {blocked_ip}")
                    block_ip(blocked_ip)
        time.sleep(30)

# Firewall and Sniffing Initialization
if __name__ == "__main__":
    sniff_thread = threading.Thread(target=start_sniffing)
    sniff_thread.start()

    detect_thread = threading.Thread(target=monitor_and_block_ddos)
    detect_thread.start()

    sniff_thread.join()
    detect_thread.join()